In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

# OPEN AND LOGIN TWITTER/ X

In [2]:
driver = webdriver.Firefox()

In [3]:
driver.get('https://x.com/login')

In [4]:
username = input()


In [5]:
import getpass

passTxt = str(getpass.getpass())

In [6]:
input_username = driver.find_element(By.TAG_NAME, "input")
input_username.clear()
input_username.send_keys(username)

In [7]:
signin = driver.find_elements(By.XPATH, "//div[@role = 'button']")

In [8]:
signin[2].click()

In [9]:
password = driver.find_elements(By.TAG_NAME, "input")

In [10]:
password[1].clear()
password[1].send_keys(passTxt)

In [11]:
submit = driver.find_elements(By.XPATH, "//div[@role = 'button']")

In [12]:
submit[2].click()

# CRAWLING COMMUNITY MEMBERS

In [42]:
import time
# get community member
driver.get('https://x.com/i/communities/1779743686756299180/members')
last_height = driver.execute_script("return document.body.scrollHeight")
i = 0
members_dict = {"name":[], "username":[]}

while (True):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    members = driver.find_elements(By.XPATH, "//div[@data-testid = 'cellInnerDiv']")
    print(len(members))
    for m in members:
        if("@" in m.text):
            members_dict["name"].append(m.text.splitlines()[0])
            if(len(m.text.splitlines()) == 4):
                 members_dict["username"].append(m.text.splitlines()[2])
            else:
                 members_dict["username"].append(m.text.splitlines()[1])
            print(m.text.splitlines())
            print("--------")
        # Calculate new scroll height and compare with last scroll height.
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
            break

    last_height = new_height

print("Finished Crawling Members List")

21
['Binus Fess', 'Admin', '@binusmenfess', 'Ikuti']
--------
['r', '@piledriverwultz', 'Ikuti']
--------
['anakpantai', '@morningandcat', 'Ikuti']
--------
['aten', 'Mod', '@atenkatroz', 'Ikuti']
--------
['juannn', '@juanwmv', 'Ikuti']
--------
['aiko', '@aj05j8', 'Ikuti']
--------
['aul | Freebies TDS3', '@jungauli', 'Ikuti']
--------
['b', '@iambirubi', 'Ikuti']
--------
['Archie', '@ArchieRawr', 'Ikuti']
--------
['vieee', 'Mod', '@strawb3ery', 'Ikuti']
--------
['kevinsfඞ', '@krvinSFF', 'Ikuti']
--------
['clara de sforza', '@clarazula', 'Ikuti']
--------
['(willsens)', '@Willsens2', 'Ikuti']
--------
['yasun', '@mocsunJA', 'Ikuti']
--------
['saras', '@dapsrst', 'Ikuti']
--------
['rere 𖤐⭒๋࣭ ⭑', '@tiaphariteler', 'Ikuti']
--------
['Vyn', '@munyamunya88', 'Ikuti']
--------
['Rene', '@iessthers', 'Ikuti']
--------
['a', '@XXenatha', 'Ikuti']
--------
['kani', '@lowdydoo', 'Ikuti']
--------
41
['Binus Fess', 'Admin', '@binusmenfess', 'Ikuti']
--------
['r', '@piledriverwultz', 'Ik

In [43]:
df = pd.DataFrame(members_dict)

In [44]:
df

,name,username
0,Binus Fess,@binusmenfess
1,r,@piledriverwultz
2,anakpantai,@morningandcat
3,aten,@atenkatroz
4,juannn,@juanwmv
...,...,...
377,grace,@antifragrace
378,fad,@jenamourrr
379,@aventurine_wz,Ikuti
380,saa pi mo mo,@snowlyie


In [45]:
df.to_csv("binus_fess_members.csv",index=False)

In [28]:
len(members)

103

In [ ]:
divs = driver.find_elements(By.XPATH,"//div[@dir='ltr']")

# CRAWLING TWEET

In [31]:
# driver.get('https://x.com/sweetersner')

def is_protected():
    import time
    time.sleep(2)
    divs = driver.find_elements(By.XPATH,"//div[@dir='ltr']")
    if len(divs) > 0:
        divs_all = [div.text if str(div.text) else "skip" for div in divs]
        if("Postingan ini dilindungi" in divs_all):
            print("protected")
            return True
    return False

In [49]:
def checkPostExist():
    infopost = driver.find_elements(By.XPATH, "//div[@dir = 'ltr']")
    # print(len(infopost))
    for i in infopost:
        if(len(i.text.splitlines()) > 0):
            if "0 postingan" == i.text.splitlines()[0]:
                print(i.text)
                return False
    return True
    

In [44]:
def import_data():
    import json
    import os
    json_file_path = "craw_data.json"
    datas = {}
    if os.path.exists(json_file_path):
        with open(json_file_path, 'r') as json_file:
            # Step 5: Read the JSON file and convert it to a dictionary
            datas = json.load(json_file)
        print(datas.keys())
    else:
        print("File does not exist. new dictionary created")
        datas = {}
    return datas

In [45]:
def save_dict(datas):
    import json

    # Convert the dictionary to a JSON string
    json_data = json.dumps(datas)
    print("checking keys of dictionary: ")
    print(datas.keys())
    # Save the JSON to a file
    if(len(datas.keys()) > 0):
        with open("craw_data.json", "w") as file:
            file.write(json_data)
    else:
        print("cannot save file, no data in dictionary")

In [46]:
def crawl(datas,x_account, limit, with_replies):
    import time
    if not with_replies:
        driver.get('https://x.com/'+x_account)
    else:
        driver.get('https://x.com/'+x_account+"/with_replies")
    protected = False
    try:
        protected = (is_protected() or (not checkPostExist()))
    except:
        import sys
        print("Error")
        save_dict(datas)
        sys.exit()
    if(protected):
        print("protected or has no post")
        return False, True
    # Get scroll height.
    last_height = driver.execute_script("return document.body.scrollHeight")
    i = 0
    while (i < limit):

        # Scroll down to the bottom.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load the page.
        time.sleep(5)
        tweet = driver.find_elements(By.XPATH, "//div[@data-testid = 'tweetText']")
        print(len(tweet))
        if(len(tweet) == 0):
            print("============== no tweet found, return to account loop =============")
            return False, False
        
        for t in tweet:
            if i > limit:
                break
            print(t.text)
            i+=1
            if not with_replies:
                if t.text not in datas[x_account]["post"]:
                    datas[x_account]["post"].append(t.text.lower())
            else:
                if t.text not in datas[x_account]["replies"]:
                    datas[x_account]["replies"].append(t.text.lower())
            print("-----------------------------")
        # Calculate new scroll height and compare with last scroll height.
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:

            break

        last_height = new_height
    return True, False
        

In [47]:
datas = import_data()
skip = False
crawl_count = user_count = 0
# users = pd.read_csv("users.csv")["username"].tolist()
users = pd.read_csv("binus_fess_members.csv")["username"].tolist()

dict_keys(['@yeoracula', '@ai_istired', '@pacarddoy', '@tatanies', '@krvinSFF', '@milkhoneysweett', '@forcevield', '@oresanna', '@jsicasthanie', '@clarazula', '@xorain_fluffy', '@vandh_', '@wittyhog', '@dreamovermorals', '@seanne_7', '@nanakimbapppp', '@sebewntin', '@parkeujust', '@nagetdinogeprek', '@hanksmumma', '@cexsaaa', '@Chapibaruyaa', '@bukangirlboss', '@piledriverwultz', '@morningandcat', '@juanwmv', '@aj05j8', '@jungauli', '@ArchieRawr', '@Willsens2', '@mocsunJA', '@dapsrst', '@tiaphariteler', '@munyamunya88', '@iessthers', '@lowdydoo', '@XXenatha', '@lanamaulanna', '@akuanakhooman', '@martabakejoe', '@anakomaliong', '@enatdeh', '@capekbgt124', '@gensetterz', '@gabbinhere', '@seericheese', '@geez_ann_', '@polayoid', '@DuncanDonut_', '@hewhoflauntsred', '@aikiropass', '@nishimiow', '@ROisinitial', '@mumbocutie', '@peterrtingle', '@Wh1zKt', '@aizzzzh', '@BaronBram', '@sasalelah', '@tunaable', '@electnibro', '@DlMABUK', '@lesyeuxdeasaa', '@GtMiyye', '@kaiinotherern', '@baratgasw

In [48]:
for x_account in users:
    import time
    limit = 200
    print(f"crawling: {x_account}")
    if x_account not in datas.keys() or ((len(datas[x_account]["post"]) == 0) and (len(datas[x_account]["replies"]) == 0)):
        datas[x_account] = {}
        datas[x_account]["post"] = []
        datas[x_account]["replies"] = []
        protected = False
        print("---------- crawling post -----------------")
        cond1, protected = crawl(datas, x_account, limit, False)
        if protected:
            print(f"skipping user: {x_account}")
            datas.pop(x_account)
            skip = True
            continue
        print("---------- crawling replies -----------------")
        cond2, protected = crawl(datas, x_account, limit, True)
        if(not cond1):
            print("!! ----- 6 minute sleep to retry post twitter view count ----- !!")
            time.sleep(360)
            cond1 = crawl(datas, x_account, limit, False)
            skip = False
        if(not cond2):
            print("!! ----- 6 minute sleep to retry replies twitter view count ----- !!")
            time.sleep(360)
            cond2 = crawl(datas, x_account, limit, True)
            skip = False
        cond = cond1 and cond2
        if cond:
            time.sleep(10)
            skip = False
            crawl_count+=1
        else:
            print("saving data because of twitter error")
            datas.pop(x_account)
            save_dict(datas)
            break
        # if cond:
        #     print("waiting to move next account")
        #     if((crawl_count+1) % 3 == 0) and not skip:
        #         print("!! ----- 5 minute sleep to trick twitter view count ----- !!")
        #         time.sleep(360)
        #         skip = False
        #         crawl_count+=1
        #     else:
        #         time.sleep(10)
        #         skip = False
        #         crawl_count+=1

    else:
        print(f"skipping user: {x_account}")
        skip = True
        continue
    user_count+=1
print("saving all data")
save_dict(datas)

crawling: @binusmenfess
---------- crawling post -----------------
13
Nilai keluar dimulai dari tanggal 7 Mei 2024, kalau ada nilai yg belum keluar pada hari itu harap ditungu 1-2 minggu kemudian.
-----------------------------
Binus! Ini jadwal keluarnya nilai uts yaa
-----------------------------
Hello B28! Catat tanggal-tanggal penting ini yaa!!
-----------------------------
kalian kalo pup di kampus gimana deh?
-----------------------------
binus! yuhuu aloha
-----------------------------
binus! woy elon cenbir gw mana >:(
-----------------------------
binus! tes
-----------------------------
binus! pembayaran bp 3 itu apa ya?
-----------------------------
[FP-FL ALERT]
-----------------------------
besok senin nder
-----------------------------
sapa anjir yg nyiptain soal kek gt
-----------------------------
good luck ujiannya fess
-----------------------------
nunggu cenbir lama bgt ya..
-----------------------------
23
gib awey
-----------------------------
Gais, menyambut uts ak

KeyboardInterrupt: 

In [ ]:
import pandas as pd
df = pd.DataFrame(datas)
df.to_csv("datas.csv")